Date : 16/12/2023

## CNN for Sentiment Analysis

<img src="" alt="">

In [1]:
!pip install nltk

!pip install keras_preprocessing

In [6]:
!pip install keras_preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 759.0 kB/s eta 0:00:00


!pip install keras preprocessing

In [3]:
!pip  install nltk --upgrade

In [4]:
from nltk.util import acyclic_depth_first as acyclic_tree

In [7]:
### --------------------
### Importing Librarires
### --------------------

import string
import re


from os import listdir


from keras.preprocessing.text import Tokenizer
from keras.utils import plot_model
from keras.models import Sequential, load_model
from keras.layers import Dense, Flatten, Embedding, Conv1D, MaxPooling1D
from keras_preprocessing.sequence import pad_sequences


from sklearn.metrics import accuracy_score


import numpy as np
from numpy import array

import nltk



In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

### Defining the function to read files one by one

In [9]:
# load doc into memory

def load_doc(filename):

    # open the file as read only
    file = open(filename, 'r')

    #read all text
    text = file.read()

    # close the file
    file.close()

    return text

In [10]:
# tetsting the 'load_doc' function

# reading a random file from the directory

text = load_doc('review_polarity/txt_sentoken/pos/cv026_29325.txt')

text

FileNotFoundError: ignored

In [ ]:
#Turn a doc into clean token
def clean_doc(doc):
    #Split ino tokens by white space
    tokens=doc.split()
    #Prepare regax for char filtering
    re_punc=re.compile('[%s]' % re.escape(string.punctuation))
    #remove punctuation from each words
    tokens=[re_punc.sub('',w)for w in tokens]
    #remove remaining tokens that are not alphabetic
    tokens=[word for word in tokens if word.isalpha()]
    #filter out stop words
    stop_words=set(stopwords.words('english'))
    tokens=[w for w in tokens if not w in stop_words]
    #Filter out short tokens
    tokens=[word for word in tokens if len(word)>1]
    return tokens

In [11]:
clean=clean_doc(text)
len(clean)

NameError: ignored

In [ ]:
# load doc, clean and return line of tokens
def doc_to_line(filename,vocab):
    #load the doc
    doc=load_doc(filename)
    #clean doc
    tokens=clean_doc(doc)
    #Filter the vocab
    tokens=[w for w in tokens if w in vocab]
    return ' '.join(tokens)

In [12]:
vocab=open('vocab.txt')
vocab=vocab.read().split()

FileNotFoundError: ignored

In [ ]:
doc_to_line('review_polarity/txt_sentoken/pos/cv026_29325.txt',vocab)

In [ ]:
#Seprating traing and test data
def process_train(directory,vocab):
    documents=list()
     # to check files in directory
    for filename in listdir(directory):
        if not filename.startswith('cv9'):
            path=directory + '/'+filename
            doc=load_doc(path)
            tokens=clean_doc(doc)
            documents.append(tokens)
    return documents


# Test data
def process_test(directory,vocab):
    documents=list()
     # to check files in directory
    for filename in listdir(directory):
        if filename.startswith('cv9'):
            path=directory + '/'+filename
            doc=load_doc(path)
            tokens=clean_doc(doc)
            documents.append(tokens)
    return documents

### Function to read all the docs

In [13]:
def process_docs(directory,vocab,is_train):
    documents = list()
    #walks through all the files in the folder
    for filename in listdir(directory):
        #skip any reviews in test set
        if is_train and filename.startswith('cv9'):
            continue
        if not is_train and not filename.startswith('cv9'):
            continue
        #create a full path of the file to open
        path = directory + '/'+filename
        #load the doc
        doc = load_doc(path)
        #clean the data
        tokens = clean_doc(doc)
        #load the data
        documents.append(tokens)
    return documents

In [14]:
lines = process_docs('review_polarity/review_polarity/txt_sentoken/pos', vocab, False)

len(lines)

NameError: ignored

In [15]:
# load and clean a dataset


def load_clean_dataset(vocab, is_train):

    # load documents
    neg = process_docs('review_polarity/review_polarity/txt_sentoken/neg',
                      vocab, is_train)

    pos = process_docs('review_polarity/review_polarity/txt_sentoken/pos',
                      vocab, is_train)

    docs = neg + pos

    # prepare labels

    labels = [0 for _ in range(len(neg))] + [1 for _ in range(len(pos))]

    return docs, labels

In [16]:
train, train_labels = load_clean_dataset(vocab, True)
test, test_labels = load_clean_dataset(vocab, False)

NameError: ignored

In [17]:
len(train), len(train_labels)

NameError: ignored

In [ ]:
len(test), len(test_labels)

## Build ANN model

In [ ]:
# define the model



def define_model(n_words):



    # define network

    model = Sequential()

    model.add(Dense(50,
                   input_shape = (n_words,),
                   activation = 'relu'
                   )
             )

    model.add(Dense(1,
                   activation = 'sigmoid'
                   )
             )



    # compile network

    model.compile(loss = 'binary_crossentropy',
                 optimizer = 'adam',
                 metrics = ['accuracy']
                 )



    # summarize the defined model

    model.summary()




    # Plotting the model

    plot_model(model,
              to_file = 'model.png',
              show_shapes = True
              )



    return model

In [ ]:
# model call

define_model(100)

In [ ]:
# fit a tokenizer

def create_tokenizer(lines):

    tokenizer = Tokenizer()

    tokenizer.fit_on_texts(lines)

    return tokenizer



# Create the tokenizer

tokenizer = create_tokenizer(train)

In [ ]:
# integer encode and pad documents

def encode_docs(tokenizer, max_length, docs):

    #integer encode
    encoded = tokenizer.texts_to_sequences(docs)

    # pad sequence
    padded = pad_sequences(encoded, maxlen = max_length, padding = 'post')

    return padded

In [ ]:
# encode data

x_train = tokenizer.texts_to_matrix(train,
                                   mode = 'binary')

x_train.shape

In [ ]:
x_test = tokenizer.texts_to_matrix(test,
                                   mode = 'binary')

x_test.shape

In [ ]:
tokenizer.word_docs

In [ ]:
#define network

n_words=x_train.shape[1]

model=define_model(n_words)

In [ ]:
model.fit(x_train,np.array(train_labels),batch_size=10,epochs=10)

In [ ]:
# Plotting the architecture of the model

plot_model(model,
           show_dtype = True,
          show_layer_activations = True,
          show_shapes = True,
          show_layer_names = True)

In [ ]:
model.evaluate(x_test, np.array(test_labels), batch_size = 1)

In [ ]:
text11 = 'Best movie ever! It was great, I will definitely recommend it.'
text22 = 'This is a bad movie'

In [ ]:
def predict_sentiment(review):

    #clean
    token = clean_doc(review)

    #convert to line
    line = ' '.join(token)

    #encode
    encoded = tokenizer.texts_to_matrix((line), mode = 'binary')

    #predict sentiment
    yhat = model.predict(encoded, verbose = 0)

    # retrieve predicted percentage and label
    percent_pos = yhat[0,0]

    if round(percent_pos) == 0:

        return (1 - percent_pos), 'NEGATIVE'

    return percent_pos, 'POSITIVE'


In [ ]:
percentage, sentiment=predict_sentiment(text22)
print(percentage,sentiment)

In [ ]:
predict_sentiment(text11)